# ASP工具箱教程1：从文件导入XS脚本到场景
### 攥写：<font color="grey">[babycat](https://github.com/babycat262)</font>        技术支持：<font color="grey">[Alian713](https://github.com/Divy1211)</font>

    在对帝国2战役进行设计时，有时需要将一些XS函数定义在场景内部以供后续调用，可以借助Python的 AoE2ScenarioParser工具包（以下称ASP），从外部文件导入XS脚本（并非从“地图”选项卡引用外部XS文件）。

场景加载XS脚本的方式有3种：
 - 1.在“地图”选项卡中脚本文件文本框，填写外部XS文件的名称（不含扩展名），引用脚本。
 - 2.手动将XS函数定义到场景内。(条件：脚本调用 / 效果：脚本调用)
 - <font color="green">3.借助ASP，从 .xs文件导入XS脚本到场景中。</font>
 - <font color="red">【注意】XS函数的定义（导入）触发器位置，应该在所有调用它的触发器之前。遵循先定义后调用的原则。</font>

前2种方式在文档《关于地编触发“脚本调用”的使用说明》.pdf 中已经做了介绍，此处将重点讲解第3种方式，利用ASP导入脚本到场景中。

## 第一部分：从文件中导入XS基本功能 和自定义XS功能

#### 步骤如下：
 - 1.在地图编辑器中创建一个场景（此处有一个名为 **Sanatorium.aoe2scenario**的场景，已经画好地形图）  
<img src="./images/doc-101.png" width=800 height=300 align="center">

 - 2.用ASP读取场景，导入外部XS文件中的函数（存放于xs目录下的**xs_func_mini.xs**，）该脚本中包含了[UGC网站中的XS基本函数](https://ugc.aoe2.rocks/general/xs/functions/)，以及一些常用自定义函数：  
<img src="./images/doc-102.png" align="center">


- 3. 安装AoE2ScenarioParser，并加载场景
```bash
# 在终端执行以下命令安装 AoE2ScenarioParser
$ pip install AoE2ScenarioParser
```

In [1]:
# 导入ASP库
from AoE2ScenarioParser.scenarios.aoe2_de_scenario import AoE2DEScenario
# 从utils中导入工具函数，实现对文件脚本的读取
from utils import load_const, load_function

In [2]:
# 设置玩家基本信息
infos = {
    "user": "babycat",
    "game_id": 76561198386517457,
}

# 输入/输出 场景名称
input_scx = "Sanatorium"
output_scx= "Sanatorium [XS]"
# 输入，输出路径
src_path = f"C:/Users/{infos['user']}/Games/Age of Empires 2 DE/{infos['game_id']}/resources/_common/scenario/{input_scx}.aoe2scenario"
des_path = f"C:/Users/{infos['user']}/Games/Age of Empires 2 DE/{infos['game_id']}/resources/_common/scenario/{output_scx}.aoe2scenario"
# des_path = f"./outputs/{output_scx}.aoe2scenario"

In [3]:
print(src_path)
print(des_path)

C:/Users/babycat/Games/Age of Empires 2 DE/76561198386517457/resources/_common/scenario/Sanatorium.aoe2scenario
C:/Users/babycat/Games/Age of Empires 2 DE/76561198386517457/resources/_common/scenario/Sanatorium [XS].aoe2scenario


In [4]:
# 读取场景
scenario = AoE2DEScenario.from_file(src_path)
# 使用触发管理器查看所有触发
trigger_mgr = scenario.trigger_manager
print("原场景触发器数量： ", len(trigger_mgr.triggers))


[11:25:28] Reading file: 'C:/Users/babycat/Games/Age of Empires 2 DE/76561198386517457/resources/_common/scenario/Sanatorium.aoe2scenario'
[11:25:28] Reading scenario file finished successfully.

############### Attributes ###############
>>> Game version: 'DE'
>>> Scenario version: 1.53
>>> Scenario variant: 'Age of Empires 2'
##########################################

[11:25:28] Loading scenario structure finished successfully.
[11:25:28] Parsing scenario file...
	✔ FileHeadereHeader data...
	✔ DataHeaderaHeader data...
	✔ Messagesessages data...
	✔ Cinematicsematics data...
	✔ BackgroundImagendImage data...
	✔ PlayerDataTwoDataTwo data...
	✔ GlobalVictoryVictory data...
	✔ Diplomacyplomacy data...
	✔ OptionsOptions data...
	✔ Maping Map data...
	✔ Unitsg Units data...
	✔ Triggersriggers data...
	✔ Filesg Files data...
[11:25:33] Parsing scenario file finished successfully.

[11:25:33] Setting up managers ...
	✔ MessageManagereManager...
	✔ PlayerManagerrManager...
	✔ MapManagerpMa

#### 从以上日志信息看到场景已经成功读取，且trigger_mgr中的触发数量是0（空列表）。接下来要给场景新建触发器，并从XS文件导入函数到场景中。

In [5]:
# （可选）创建标题触发，为你的场景/战役起个名字
title_trigger = trigger_mgr.add_trigger("*****《毛利人的村庄》*****", enabled=False)
# 查看触发名称
print(title_trigger.name)

*****《毛利人的村庄》*****


In [6]:
# 新建触发器，名称为XS_FUNC_TITLE所指定的名称，该触发器存放XS定义
# 【注意】将XS脚本作为条件定义导入场景中，该触发器状态必须要关闭，要设置 enabled=False
XS_FUNC_TITLE= r"XS函数定义"
fn_trigger = trigger_mgr.add_trigger(XS_FUNC_TITLE, enabled=False)
print(fn_trigger.name)

XS函数定义


In [7]:
# 用函数 load_function 读取 XS文件内容到列表中
_, fn_list = load_function("./xs/xs_func_mini.xs", sep='\n\n\n')
fn_list

['const float EPS=0.000001;const float PI=3.1416;const float E_=2.7183;const int Inf=32768;',
 'void _EA(int mode=-1,int id_=0,int attr=-1,float val_=0.0,int p=-1){if(mode>=0 && mode<=9){xsEffectAmount(mode,id_,attr,val_,p);}}',
 'void _Chat(string msg="",string color="",float val_=-32768){if(val_!=-32768){msg=color+msg+": "+val_;} else{msg=color+msg;} xsChatData(msg);}',
 'int _BoolArr(int arr_len=1,bool val_=false,string arr_name=""){return (xsArrayCreateBool(arr_len,val_,arr_name));}',
 'int _IntArr(int arr_len=1,int val_=0,string arr_name=""){return (xsArrayCreateInt(arr_len,val_,arr_name));}',
 'int _FloatArr(int arr_len=1,float val_=0.0,string arr_name=""){return (xsArrayCreateFloat(arr_len,val_,arr_name));}',
 'int _StrArr(int arr_len=1,string val_="",string arr_name=""){return (xsArrayCreateString(arr_len,val_,arr_name));}',
 'int _VecArr(int arr_len=1,vector val_=cOriginVector,string arr_name=""){return (xsArrayCreateVector(arr_len,val_,arr_name));}',
 'bool _BArrG(int arr=-1,

#### 现在已经将XS文件中的内容读取到列表`fn_list`中。接着遍历该列表，把每个元素（XS功能）定义到场景中。

In [8]:
# 遍历 fn_list将其中每一项定义到场景
for fn in fn_list:
    fn_trigger.new_condition.script_call(fn)
print("XS函数基本导入成功！")

XS函数基本导入成功！


In [9]:
# 查看触发中的XS脚本导入情况
fn_trigger.conditions

[Condition: 	{
 		_instance_number_history: 	[]
 		_uuid: c161100d-3007-4cb9-a4ab-bf7cf305f833
 		condition_type: SCRIPT_CALL
 		quantity: -1
 		attribute: -1
 		unit_object: -1
 		next_object: -1
 		object_list: -1
 		source_player: -1
 		technology: -1
 		timer: -1
 		area_x1: -1
 		area_y1: -1
 		area_x2: -1
 		area_y2: -1
 		object_group: -1
 		object_type: -1
 		ai_signal: -1
 		inverted: -1
 		variable: -1
 		comparison: -1
 		target_player: -1
 		unit_ai_action: -1
 		object_state: -1
 		timer_id: -1
 		victory_timer_type: -1
 		include_changeable_weapon_objects: -1
 		xs_function: const float EPS=0.000001;const float PI=3.1416;const float E_=2.7183;const int Inf=32768;
 	},
 Condition: 	{
 		_instance_number_history: 	[]
 		_uuid: c161100d-3007-4cb9-a4ab-bf7cf305f833
 		condition_type: SCRIPT_CALL
 		quantity: -1
 		attribute: -1
 		unit_object: -1
 		next_object: -1
 		object_list: -1
 		source_player: -1
 		technology: -1
 		timer: -1
 		area_x1: -1
 		area_y1: -1
 		area_x2:

从输出结果可以看出，basefn_trigger 的条件列表中，对应的 `xs_function`属性都正确赋以了一个XS函数脚本，如图所示：  
<img src="./images/doc-103.png" width=300 height=800>

In [10]:
# 将修改后的场景进行保存
scenario.write_to_file(des_path)


[11:27:31] Reconstructing sections and structs from managers...
	✔ MessageManagerssageManager...
	✔ PlayerManagerlayerManager...
	✔ MapManagerg MapManager...
	✔ UnitManager UnitManager...
	✔ TriggerManageriggerManager...
	✔ XsManagerng XsManager...
	✔ OptionManagerptionManager...
[11:27:33] Reconstruction finished successfully.

[11:27:33] File writing from structure started...
	✔ FileHeaderg FileHeader...
	✔ DataHeaderg DataHeader...
	✔ Messagesing Messages...
	✔ Cinematicsg Cinematics...
	✔ BackgroundImagekgroundImage...
	✔ PlayerDataTwolayerDataTwo...
	✔ GlobalVictorylobalVictory...
	✔ Diplomacyng Diplomacy...
	✔ Optionsting Options...
	✔ Maptructing Map...
	✔ Unitsucting Units...
	✔ Triggersing Triggers...
	✔ Filesucting Files...
[11:27:34] File writing finished successfully.
[11:27:34] File successfully written to: 'C:/Users/babycat/Games/Age of Empires 2 DE/76561198386517457/resources/_common/scenario/Sanatorium [XS].aoe2scenario'
[11:27:34] Execution time from scenario read: 12

## 第二部分：从文件中导入用户定义(UDF)函数
    用户自定义的UDF函数，可以实现除了基础函数外的特定功能。这些函数能够实现更灵活的功能定制。
假设现在有几个已实现的UDF函数：
 - `enable_unit`: 启用指定单位与某个建筑中
 - `unit_blast`：设置单位践踏
 - `unit_charges`：设置单位蓄力
 - `unit_combats`：设置单位作战能力
 - `unit_projectiles`：设置单位抛掷物
 - `proj_smart_mode`：设置抛掷物智能模式
 - `units_transform`：单位形态转换（单位变身）


#### 下面是一个UDF函数的示例：
```c++
/**
 * @brief 设置单位蓄力
 * @param unit: 单位ID
 * @param charge_event: 蓄力事件。 {0:"无蓄力", 1:"有蓄力"}
 * @param charge_type: 蓄力类型。 {0:"无蓄力攻击，不能闪避抛射物", 1:"蓄力攻击", 3:"蓄力范围攻击", 4:"闪避抛射物"}
 * @param max_charge: 最大蓄力值，单位拥有的最大蓄力充能点数。 最大蓄力值 = 蓄力再生率*蓄力CD
 * @param recharge_rate : 蓄力再生速率(点/秒)
 * @param p: 玩家序号
 * @param charge_cd: 蓄力充能周期（秒）。若该值设置成大于0的数值，则蓄力再生速率由以下公式确定：蓄力再生速率 = 最大蓄力值/蓄力CD
 * @return <Bool> 若蓄力成功设置返回true，否则返回false
**/
bool unit_charges(int unit=0, int charge_event=0, int charge_type=0, float max_charge=0.0, float recharge_rate=0.0, int p=-1, float charge_cd=-1) 
{
    if(charge_event==0 || (charge_type<0 || charge_type>4 || charge_type==2)) {return (false);}
    if(charge_cd > 0) {recharge_rate = max_charge/charge_cd;}
    xsEffectAmount(0, unit, 61, charge_event, p);
    xsEffectAmount(0, unit, 62, charge_type, p);
    xsEffectAmount(0, unit, 59, max_charge, p);
    xsEffectAmount(0, unit, 60, recharge_rate, p);
    return (true);
}
```

这些函数已经在`UDF模块`中实现，在下面的单元格中将它导入：

In [11]:
#改变输出场景名称（可选）
des_path = des_path.replace('[XS]', '[XS_ALL]')
print(des_path)

C:/Users/babycat/Games/Age of Empires 2 DE/76561198386517457/resources/_common/scenario/Sanatorium [XS_ALL].aoe2scenario


In [12]:
# 从文件种加载UDF函数列表
udf_list = load_function("./xs/hero_skills.xs")[1]
udf_list

['void unit_blast(int unit=-1,int blast_att=-1,int blast_def=-1,float blast_w=0.0,float blast_rate=0.0,int p=-1) {if(blast_att==0||blast_att==1||blast_att==2||blast_att==4||blast_att==64||blast_att==128) {xsEffectAmount(0,unit,44,blast_att,p);}if(blast_def==0||blast_def==1||blast_def==2||blast_def==4) {xsEffectAmount(0,unit,45,blast_def,p);}if(blast_w > 0.0) {xsEffectAmount(0,unit,22,blast_w,p);}if(blast_rate > 0.0 && blast_rate <= 1.0) {xsEffectAmount(0,unit,115,blast_rate,p);}else if(blast_rate < 0 && blast_rate > 0-255) {xsEffectAmount(0,unit,115,1*blast_rate,p);}}',
 'bool unit_charges(int unit=-1,int charge_event=0,int charge_type=0,float max_charge=0.0,float recharge_rate=0.0,int p=-1,float charge_cd=-1) {if(charge_event==0||(charge_type<0||charge_type>4||charge_type==2)) {return (false);}if(charge_cd > 0) {recharge_rate=max_charge/charge_cd;}xsEffectAmount(0,unit,61,charge_event,p);xsEffectAmount(0,unit,62,charge_type,p);xsEffectAmount(0,unit,59,max_charge,p);xsEffectAmount(0,un

In [13]:
# 在名为 XS_FUNC_TITLE 的触发器条件中追加定义 UDF的XS功能
for trigger in trigger_mgr.triggers:
    if trigger.name == XS_FUNC_TITLE:
        # 将UDF导入场景
        for fn in udf_list:
            trigger.new_condition.script_call(fn)
print("UDF函数导入成功！")

UDF函数导入成功！


In [14]:
# 查看triggers列表中UDF添加情况
trigger_mgr.triggers[1].conditions

[Condition: 	{
 		_instance_number_history: 	[
 			1, 0
 		]
 		_uuid: c161100d-3007-4cb9-a4ab-bf7cf305f833
 		condition_type: SCRIPT_CALL
 		quantity: -1
 		attribute: -1
 		unit_object: -1
 		next_object: -1
 		object_list: -1
 		source_player: -1
 		technology: -1
 		timer: -1
 		area_x1: -1
 		area_y1: -1
 		area_x2: -1
 		area_y2: -1
 		object_group: -1
 		object_type: -1
 		ai_signal: -1
 		inverted: -1
 		variable: -1
 		comparison: -1
 		target_player: -1
 		unit_ai_action: -1
 		object_state: -1
 		timer_id: -1
 		victory_timer_type: -1
 		include_changeable_weapon_objects: -1
 		xs_function: const float EPS=0.000001;const float PI=3.1416;const float E_=2.7183;const int Inf=32768;
 	},
 Condition: 	{
 		_instance_number_history: 	[
 			1, 1
 		]
 		_uuid: c161100d-3007-4cb9-a4ab-bf7cf305f833
 		condition_type: SCRIPT_CALL
 		quantity: -1
 		attribute: -1
 		unit_object: -1
 		next_object: -1
 		object_list: -1
 		source_player: -1
 		technology: -1
 		timer: -1
 		area_x1: -1


#### 从以上结果看到，在原先定义的XS基本函数的后面，又追加定义了UDF的XS功能。

In [15]:
#脚本导入成功后，将场景进行保存
scenario.write_to_file(des_path)


[11:38:34] Reconstructing sections and structs from managers...
	✔ MessageManagerssageManager...
	✔ PlayerManagerlayerManager...
	✔ MapManagerg MapManager...
	✔ UnitManager UnitManager...
	✔ TriggerManageriggerManager...
	✔ XsManagerng XsManager...
	✔ OptionManagerptionManager...
[11:38:35] Reconstruction finished successfully.

[11:38:35] File writing from structure started...
	✔ FileHeaderg FileHeader...
	✔ DataHeaderg DataHeader...
	✔ Messagesing Messages...
	✔ Cinematicsg Cinematics...
	✔ BackgroundImagekgroundImage...
	✔ PlayerDataTwolayerDataTwo...
	✔ GlobalVictorylobalVictory...
	✔ Diplomacyng Diplomacy...
	✔ Optionsting Options...
	✔ Maptructing Map...
	✔ Unitsucting Units...
	✔ Triggersing Triggers...
	✔ Filesucting Files...
[11:38:36] File writing finished successfully.
[11:38:36] File successfully written to: 'C:/Users/babycat/Games/Age of Empires 2 DE/76561198386517457/resources/_common/scenario/Sanatorium [XS_ALL].aoe2scenario'
[11:38:36] Execution time from scenario read

#### 我们用地图编辑器打开导出的场景，切换到“触发”选项卡，可以看到XS脚本已经成功导入到场景里了。

<img src="./images/doc-104.png" width=600 height=900 align="center">

### <font color="red">ASP导入XS脚本的优势：</font>
#### 相比手动在场景中添加XS脚本，ASP导入脚本有以下几个优势：
 - <font color="blue">**批量导入：ASP可以将文件中定义好的脚本批量导入场景，省时省力**</font>
 - <font color="blue">**修改方便：若要修改XS函数定义，只需将修改后的文件进行导入，覆盖原先的定义即可（再次导入前，先将XS所在触发的条件列表清空：`trigger_mgr.triggers[1].conditions=[]`）**（此处triggers[1]触发用于定义脚本）</font>
 - <font color="blue">**导入脚本的字符个数不受限制（若手动向条件脚本调用中添加内容，脚本字符个数不能超过256）**</font>
